In [1]:
import pandas as pd
from Datos import Ultimate_df
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

c:\Users\EQUIPO\Documents\PIH_ML\Datos.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  Ultimate_df= pd.read_csv('../PIH_ML/Final-Datasets/datos-transformados.csv')


Despues de importa las librerias a utilizar y la base de datos a trabajar seleccionamos las columnas a utilizar

In [2]:

Ultimate_df['title']=Ultimate_df['title'].str.replace(r"\W"," ")

Ultimate_df.drop_duplicates(inplace=True)

C:\Users\EQUIPO\AppData\Local\Temp\ipykernel_1380\411658153.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Ultimate_df['title']=Ultimate_df['title'].str.replace(r"\W"," ")


Al tener archivos repetidos volvemos a definir el promedio de rating

In [3]:
item_ml=Ultimate_df[['title','vote_average','vote_count']]
item_ml['count']=1
item_ml=item_ml.groupby('title').sum()

C:\Users\EQUIPO\AppData\Local\Temp\ipykernel_1380\3513213634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_ml['count']=1


In [4]:
item_ml['rating_avg']=item_ml['vote_average']/item_ml['count']

In [5]:
item_ml=item_ml[['vote_count','rating_avg']]

In [6]:
item_ml['title']=item_ml.index
item_ml.reset_index(drop=True,inplace=True)

Creamos un dataframe reducido para poder predecir mejor ya que algunas peliculas tienen muy poco numero de rating

In [7]:
item_ml=item_ml[['title','rating_avg','vote_count']]

In [8]:
item_ml_reduced=item_ml.sort_values('vote_count',ascending=False).head(1500).reset_index()[['title','rating_avg','vote_count']]

In [9]:
item_ml_reduced['title']=item_ml_reduced['title'].apply(lambda x:x.strip())

Ya esta lista la matriz para entrenar el modelo

In [10]:
item_ml_reduced

,title,rating_avg,vote_count
0,Inception,8.1,14075.0
1,The Dark Knight,7.3,12271.0
2,The Avengers,5.9,12205.0
3,Avatar,7.2,12114.0
4,Deadpool,7.4,11444.0
...,...,...,...
1495,Paranormal Activity 2,5.7,734.0
1496,Mommy,4.1,734.0
1497,Before Sunset,7.6,734.0
1498,Zack and Miri Make a Porno,6.1,734.0


Reescalamos los datos y creamos la matriz de similitud

In [11]:
scaler = MinMaxScaler()
item_ml_reduced['rating_norm'] = scaler.fit_transform(item_ml_reduced['rating_avg'].values.reshape(-1, 1))
item_ml_reduced['num_rankings_norm'] = scaler.fit_transform(item_ml_reduced['vote_count'].values.reshape(-1, 1))

In [12]:
feature_matrix = item_ml_reduced[['rating_norm', 'num_rankings_norm']].values


In [13]:
similarity_matrix = cosine_similarity(feature_matrix)


In [14]:
similarity_matrix =pd.DataFrame(similarity_matrix,columns=item_ml_reduced['title'],index=item_ml_reduced['title'])

Esta matriz la convertimos a csv y la guardamos dentro de la carpeta de 'Final-Datasets

In [15]:
from Datos import ml_matrix_df

La reimportamos para crear la funcion de la api


In [16]:
def recomendacion(titulo):
  arr=[]
  for i in ml_matrix_df[titulo].sort_values(ascending=False).head(6).index[1:6].to_list():
     arr.append(ml_matrix_df['title'][i])
  return arr

In [17]:
print(recomendacion('The Avengers'))

['Batman   Robin', 'Fantastic Four', 'Batman v Superman  Dawn of Justice', 'Suicide Squad', 'Avatar']
